In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from nltk import word_tokenize
from nltk.util import ngrams
from random import shuffle
import nltk
import numpy as np
import re
import os
import time

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
newspapers = os.listdir('/content/drive/MyDrive/51800553_51800182_51800563_BigData_FinalProject/Dataset/newspapers/')
print(len(newspapers))

350


**Text Cleaning**

In [ ]:
def clean_text(text):
    text = text.replace('\n', ' ')
    text = text.lower()
    text = re.sub('\W+', ' ', text)
    text = text.strip()
    
    return text

In [ ]:
def calculate_join_grams(text, n):
    words = word_tokenize(text)
    n_gram = ngrams(words, n)
    join_grams = [' '.join(list(gram)) for gram in n_gram]
    
    return join_grams

In [ ]:
def create_permutations(number_shingles, number_permutation):
    permutations = list()
    for _ in range(number_permutation):
        random_permutation_numbers = [i for i in range(1, number_shingles + 1)]
        shuffle(random_permutation_numbers)
        permutations.append(random_permutation_numbers)
        
    return permutations

create_permutations(number_shingles=5, number_permutation=3)

[[3, 4, 2, 5, 1], [2, 5, 3, 1, 4], [5, 3, 1, 2, 4]]

**Bước 1: Shingling**

In [ ]:
start_time = time.time()

newspaper_vectors = list()
shingles = list()
for index, newspaper in enumerate(newspapers):
    print(f'[{index + 1}/{len(newspapers)}] {newspaper}')
    with open(f'/content/drive/MyDrive/51800553_51800182_51800563_BigData_FinalProject/Dataset/newspapers/{newspaper}', 'r') as f:
        lines = f.readlines()
        title_and_content = lines[1] + lines[2]
        cleaned_title_and_content = clean_text(title_and_content)
        
        kgrams = calculate_join_grams(cleaned_title_and_content, 3)
        for kgram in kgrams:
            if kgram not in shingles:
                shingles.append(kgram)
        
        exist_booleans = list()
        for shingle in shingles:
            if shingle in cleaned_title_and_content:
                exist_booleans.append(True)
            else:
                exist_booleans.append(False)
                
        newspaper_vectors.append(np.array(exist_booleans, dtype=bool))

print(f'--- {time.time() - start_time} seconds ---')

[1/350] Mạng đường sắt Việt Nam đến năm 2030.txt
[2/350] Gần 500 tấn mắm ruốc tồn kho sau sự cố môi trường biển.txt
[3/350] Người dân đổi thẻ bảo hiểm y tế mới như thế nào.txt
[4/350] 25 chức danh lãnh đạo sắp được kiện toàn.txt
[5/350] Chàng trai Việt thành danh trên sân khấu opera quốc tế.txt
[6/350] 'Chống tham nhũng quyết liệt tạo ra sức cảnh tỉnh lớn'.txt
[7/350] Đường 10 làn xe trục tây Thăng Long Hà Nội.txt
[8/350] Đề xuất chặn lục bình đổ ra sông Sài Gòn.txt
[9/350] Hôm nay khai mạc kỳ họp cuối cùng, Quốc hội khóa XIV.txt
[10/350] Dự kiến khởi công cao tốc Cần Thơ - Cà Mau năm 2022.txt
[11/350] Người nhận bảo hiểm xã hội một lần ở TP HCM tăng cao.txt
[12/350] 4 ôtô tông liên hoàn khi dừng đèn đỏ.txt
[13/350] Hà Nội sẽ phát triển không gian ngầm ở 4 quận nội đô.txt
[14/350] 'Nếu ngồi xe chở gỗ,

In [ ]:
print(len(shingles))

132874


**Bước 2: MinHashing**

In [ ]:
permutations = create_permutations(number_shingles=len(shingles), number_permutation=100)

In [ ]:
start_time = time.time()

signature_vectors = list()

for index, newspaper_vector in enumerate(newspaper_vectors):
    print(f'[{index + 1}/{len(newspapers)}]', end='')
    
    signature_vector = list()
    for permutation in permutations:
        min_number = float('inf')
        for index, exist_boolean in enumerate(newspaper_vector):
            if exist_boolean:
                if permutation[index] < min_number:
                    min_number = permutation[index]
        
        signature_vector.append(min_number)
            
    signature_vectors.append(signature_vector)
    
    print(signature_vector)
    
print(f'--- {time.time() - start_time} seconds ---')
print(len(signature_vectors))

[1/350][2955, 2201, 5289, 3500, 7452, 2834, 2267, 4785, 2360, 12806, 1955, 469, 488, 1915, 1972, 3993, 9110, 2179, 20013, 1946, 840, 5234, 4031, 288, 1367, 3148, 216, 8098, 2129, 564, 3501, 3463, 5343, 4483, 12598, 1136, 1361, 3413, 5492, 10869, 1270, 451, 5290, 1846, 1885, 179, 3498, 5363, 2622, 3291, 2277, 1815, 947, 5548, 145, 3533, 7607, 12334, 2827, 19138, 1959, 718, 1834, 4948, 609, 4398, 3300, 280, 3349, 326, 985, 2822, 4051, 11043, 16741, 7070, 1804, 914, 343, 1510, 3176, 341, 279, 1126, 12517, 1363, 2352, 69, 1204, 6320, 3596, 2149, 2984, 978, 5955, 3397, 2414, 256, 1193, 1908]
[2/350][1, 111, 780, 48, 809, 564, 869, 278, 376, 441, 192, 198, 124, 125, 522, 36, 109, 11, 85, 14, 217, 64, 154, 472, 250, 127, 901, 126, 155, 413, 137, 478, 109, 18, 229, 233, 44, 91, 1001, 424, 11, 500, 64, 244, 278, 133, 55, 549, 380, 983, 46, 106, 736, 111, 226, 322, 208, 1068, 29, 16, 59, 40, 17, 67, 16, 304, 171, 56, 201, 296, 13, 181, 70, 433, 654, 9, 91, 76, 62, 220, 332, 119, 248, 352, 311, 1

**Bước 3: Thuật toán LSH**

In [ ]:
number_of_band = 20
number_of_row = int(len(permutations) / number_of_band)
print(number_of_band, number_of_row)

20 5


In [ ]:
buckets = dict()

for index_newspaper, signature_vector in enumerate(signature_vectors):
    previous_row = 0
    for _ in range(number_of_band):
        key_chars = list()
        for index in range(previous_row, previous_row + number_of_row):
            key_chars.append(signature_vector[index])
        previous_row += number_of_row

        key = '_'.join(map(str, key_chars))

        if key not in buckets:
            buckets[key] = [index_newspaper]
        else:
            buckets[key].append(index_newspaper)

print(buckets)

{'2955_2201_5289_3500_7452': [0], '2834_2267_4785_2360_12806': [0], '1955_469_488_1915_1972': [0], '3993_9110_2179_20013_1946': [0], '840_5234_4031_288_1367': [0], '3148_216_8098_2129_564': [0], '3501_3463_5343_4483_12598': [0], '1136_1361_3413_5492_10869': [0], '1270_451_5290_1846_1885': [0], '179_3498_5363_2622_3291': [0], '2277_1815_947_5548_145': [0], '3533_7607_12334_2827_19138': [0], '1959_718_1834_4948_609': [0], '4398_3300_280_3349_326': [0], '985_2822_4051_11043_16741': [0], '7070_1804_914_343_1510': [0], '3176_341_279_1126_12517': [0], '1363_2352_69_1204_6320': [0], '3596_2149_2984_978_5955': [0], '3397_2414_256_1193_1908': [0], '1_111_780_48_809': [1], '564_869_278_376_441': [1], '192_198_124_125_522': [1], '36_109_11_85_14': [1], '217_64_154_472_250': [1], '127_901_126_155_413': [1], '137_478_109_18_229': [1], '233_44_91_1001_424': [1], '11_500_64_244_278': [1], '133_55_549_380_983': [1], '46_106_736_111_226': [1], '322_208_1068_29_16': [1], '59_40_17_67_16': [1], '304_171_

In [ ]:
for _, bucket in buckets.items():
    if len(bucket) > 1:
        print(bucket)
        for index_newspaper in bucket:
            print(newspapers[index_newspaper])
        print()

[123, 132]
Một phụ nữ nhập cảnh trái phép về Hải Phòng dương tính Covid-19 Cả 19 F1 đều âm tính.txt
Hải Phòng thông báo khẩn tìm người liên quan đến 2 ca dương tính Covid-19.txt

[127, 156, 179, 186]
16.000 viên ma túy giấu trong mớ rau, vứt bên vệ đường.txt
Cận cảnh phong tỏa, khám xét khẩn cấp cây xăng tại Gò Vấp để điều tra vụ xăng giả.txt
Công an Hà Nội hoàn thành hơn 1 triệu hồ sơ thẻ căn cước công dân gắn chip.txt
Bình Dương cho mở lại các hoạt động vui chơi giải trí.txt

[127, 156, 179]
16.000 viên ma túy giấu trong mớ rau, vứt bên vệ đường.txt
Cận cảnh phong tỏa, khám xét khẩn cấp cây xăng tại Gò Vấp để điều tra vụ xăng giả.txt
Công an Hà Nội hoàn thành hơn 1 triệu hồ sơ thẻ căn cước công dân gắn chip.txt

[127, 156, 179, 186]
16.000 viên ma túy giấu trong mớ rau, vứt bên vệ đường.txt
Cận cảnh pho

In [ ]:
print(len(shingles))

132874


**Bước 4: Test thuật toán bằng cách đưa vào một bản tin bất kỳ**

In [ ]:
def get_similar_newspapers(title_and_content, threshold=0.8):
    cleaned_title_and_content = clean_text(title_and_content)

    exist_booleans = list()
    for shingle in shingles:
        if shingle in cleaned_title_and_content:
            exist_booleans.append(True)
        else:
            exist_booleans.append(False)
    
    newspaper_vector = np.array(exist_booleans, dtype=bool)
    
    signature_vector = list()
    for permutation in permutations:
        min_number = float('inf')
        for index, exist_boolean in enumerate(newspaper_vector):
            if exist_boolean:
                if permutation[index] < min_number:
                    min_number = permutation[index]
        
        signature_vector.append(min_number)
    
    same_bucket_index_newspapers = set()
    
    key_chars = list()
    previous_row = 0
    for _ in range(number_of_band):
        for index in range(previous_row, previous_row + number_of_row):
            key_chars.append(signature_vector[index])
        previous_row += number_of_row

        key = '_'.join(map(str, key_chars))

        if key in buckets:
            same_bucket_index_newspapers.update(buckets[key])
        
    similar_index_newspapers = list()
    
    for same_bucket_index_newspaper in same_bucket_index_newspapers:
        count = 0
        for i, v in enumerate(newspaper_vector):
            if i > len(newspaper_vectors[same_bucket_index_newspaper]) - 1:
                break
                
            if newspaper_vectors[same_bucket_index_newspaper][i] == v:
                count += 1
        
        jaccard_similarity = count / len(newspaper_vectors[same_bucket_index_newspaper])

        if jaccard_similarity >= threshold:
            similar_index_newspapers.append(same_bucket_index_newspaper)
            
    return similar_index_newspapers


text = '''
Xây dựng trục cảnh quan xanh, chỉnh trị dòng sông và đảm bảo sinh kế cho người dân là "ba bài toán khó nhất của quy hoạch phân khu đô thị sông Hồng".Kiến trúc sư Ngô Trung Hải, Phó chủ tịch Hội quy hoạch và phát triển đô thị Việt Nam, đưa ra nhận định trên khi trả lời phỏng vấn của VnExpress về đồ án này.- Ông nhận định như thế nào về ý nghĩa của quy hoạch phân khu đô thị sông Hồng, với diện tích lên tới 11.000 ha và trải dài 40 km?- Tôi được tham gia xây dựng quy hoạch TP Hà Nội cùng các nhà nghiên cứu nước ngoài và Viện Quy hoạch xây dựng Hà Nội, nên có thể nói đồ án này rất cần thiết để cụ thể hóa quy hoạch chung xây dựng Thủ đô đến năm 2030, tầm nhìn 2050 đã được Thủ tướng phê duyệt năm 2011; cũng là sự kết nối một chuỗi các quy hoạch phân khu khác mà TP Hà Nội đang hoàn thiện.Từ khi vua Lý Thái Tổ dời đô về Đại La, sông Hồng là một phần lịch sử của Hà Nội đến tận ngày nay. Từ lâu đời nay, đây là tuyến giao thương đường thủy chính để kinh đô buôn bán và giao lưu văn hoá với thế giới. Đến nay, ở các khu phố cổ và khu vực ven sông vẫn còn những vết tích lịch sử này.Năm 1954, Thủ đô được giải phóng và để chuẩn bị đón cán bộ, các cơ quan từ chiến khu về Hà Nội, chính quyền cách mạng đã xây dựng nhiều khu tập thể bên sông Hồng theo mô hình không khép kín, nghĩa là có khu nhà sàn, nhà ăn tập thể, khu vệ sinh và tắm rửa cũng như nhà trẻ, cửa hàng bách hoá đều riêng biệt. Có thời kỳ các khu tập thể này lên đến chục vạn hộ gia đình. Sau này, cùng với quá trình đô thị hóa, đa số người dân có điều kiện vào sống ở trung tâm thành phố. Các khu tập thể bên sông chỉ còn những người lao động nghèo, mức sống thấp. Đây cũng trở thành nơi tập kết vật liệu, cảng sông, nhà máy kho xưởng và cũng là nơi nước thải đô thị chảy ra nên nhiều người ví von rằng chúng ta sống "quay lưng" vào sông.Vì vậy, với quy mô diện tích khoảng 11.000 ha, kéo dài 40 km từ cầu Hồng Hà đến cầu Mễ Sở, ý tưởng đầu tiên của đồ án lần này là muốn tổ chức lại không chỉ không gian cảnh quan đô thị dọc hai bên bờ sông một cách quy củ, rõ ràng, mà còn đảm bảo sinh kế cho hàng chục nghìn hộ dân ở đây. Thông qua đồ án, Hà Nội sẽ "quay mặt" vào sông Hồng để phát triển.KTS Ngô Trung Hải. Ảnh: VT- Để các ý tưởng trong quy hoạch phân khu sông Hồng trở thành hiện thực, thành phố cần giải quyết những bài toán nào?- Tôi cho rằng có ba bài toán cơ bản, quan trọng nhất, nhưng cũng khó nhất được đặt ra trong đồ án cần phải giải quyết. Trước tiên, là cải tạo lại toàn bộ không gian hai bên bờ sông Hồng thành trục cảnh quan, không gian văn hóa, lá phổi xanh.Ở góc độ cá nhân, tôi rất tâm đắc với cách tiếp cận của đồ án này khi định hướng phát triển hai bên bờ sông hồng là trục cảnh quan xanh, văn hóa.Thứ hai, thực hiện được mục tiêu chỉnh trị sông Hồng, phòng chống lũ kết hợp với hệ thống giao thông dọc, ngang kết hợp các tuyến đê và cầu.Thứ ba, làm sao đảm bảo cho người dân đang sinh sống ở dọc hai bên bờ sông có nơi sống tốt hơn, ổn định hơn, hạ tầng xã hội cũng như môi trường sống được cải thiện hơn.Đồ án đặt mục tiêu xây dựng hàng loạt cây cầu nối hai bên bờ sông, cùng hệ thống giao thông dọc bờ sông, tạo ra hệ thống giao thông mới của thủ đô. Vậy nguồn lực nào để xây dựng các công trình này cũng là bài toán cần được tính toán kỹ lưỡng.- Làm sao để tìm được lời giải thích hợp cho những bài toán ấy, thưa ông?- Từ những mục tiêu lớn trong đồ án, thành phố cần xây dựng các dự án chi tiết hơn, phân tích rõ những nguồn lực để thực hiện.Chẳng hạn, thành phố có thể kêu gọi nhà đầu tư chỉnh trang lại hệ thống đê điều, xây dựng khu công viên, khu thể thao, khu nghỉ dưỡng, khu văn hóa lịch sử. Các khu vực cần được phân định rõ ràng. Nếu các dự án này được đầu tư xây dựng thành công, thì trục sông Hồng sẽ là nơi thu hút nhiều khách du lịch trong nước và quốc tế. Người dân và du khách sẽ tìm đến khu đô thị sông Hồng để nghỉ ngơi, giải trí, thưởng thức không khí trong lành và sống trong không gian lịch sử sâu lắng. Qua đó, thành phố có nguồn lực để tái đầu tư.Các dự án giao thông, nâng cấp khu dân cư, nhà cũng cần có dự án chi tiết sẽ được thực hiện như thế nào. Nhà đầu tư được mời gọi vào đây để xây dựng khu đô thị mới, nhưng phải làm sao để vẫn đảm bảo sinh kế và nơi ở cho người dân.Hay đơn giản, nếu thành phố huy động được nguồn lực để xây dựng hàng loạt cây cầu đặc sắc, ấn tượng qua sông, gắn với các địa danh hoặc mang tên nhân vật lịch sử, đó sẽ trở thành biểu tượng của thủ đô và trở thành địa điểm hấp dẫn với nhiều du khách.Bản đồ vị trí các bãi sông Hồng và cây cầu trong khu vực quy hoạch. Đồ họa: Tiến Thành- Ông tâm đắc với cách tiếp cận của đồ án khi định hướng hai bên bờ sông Hồng trở thành trục cảnh quan xanh, điều này khả thi đến đâu với xu hướng đô thị hóa hiện nay, thưa ông?- Nghe đến quy hoạch đô thị sông Hồng, có người sẽ nghĩ ngay đến việc xây dựng đô thị với các nhà cao tầng san sát hai bên bờ sông. Nhưng đồ án lần này hướng tới xây dựng trục không gian xanh dọc hai bên bờ sông.Năm 2006, các chuyên gia Hàn Quốc đã hỗ trợ Hà Nội nghiên cứu đồ án quy hoạch cơ bản hai bên bờ sông Hồng với tổng số vốn đầu tư lên đến hơn 7,1 tỷ USD. Tuy nhiên đến năm 2008, dự án quy hoạch này bị dừng triển khai. Theo tôi được biết, khi nghiên cứu quy hoạch này, các chuyên gia Hàn Quốc lấy ý tưởng muốn xây dựng đô thị sông Hồng như sông Hàn ở Seoul, là đô thị sầm uất, lên đến hàng triệu dân, với nhiều nhà cao tầng. Đây là phương án rất tham vọng, nhưng không phù hợp với Hà Nội hiện nay, bởi sau khi mở rộng, thành phố đã có nhiều không gian để xây dựng các đô thị vệ tinh, nên không cần thiết phải dồn công trình dọc hai bên sông Hồng.Nhắc lại câu chuyện trước đây để thấy rằng mục tiêu không chất tải (dồn) công trình dọc sông Hồng là khả thi. Hiện nay các khu đô thị phía Tây hay phía Bắc đang phát triển mạnh mẽ, tạo nhiều công ăn, việc làm hấp dẫn, sẽ thu hút nhiều người dân đến sinh sống. Vì vậy, giải pháp để hai bờ sông Hồng không bị chất tải là tạo thêm các khu đô thị hấp dẫn, thu hút người dân đến sinh sống, làm việc. Khi có đủ không gian thu hút người dân thì đương nhiên hai bên bờ sông Hồng sẽ đảm bảo là trục cảnh quan xanh.Sông Hồng đoạn chảy qua trung tâm Hà Nội nhìn từ trên cao. Bên phải dòng sông là quận Hoàn Kiếm và Hồ Gươm, bên trái là khu Bồ Đề (quận Long Biên). Ảnh: Ngọc Thành- Nhưng làm sao để kiểm soát việc xây dựng ở trục sông Hồng thưa ông?- Sau khi đồ án được phê duyệt, thành phố cần có các quy định chặt chẽ, minh bạch. Chính quyền các quận, huyện một mặt tạo điều kiện cho các nhà đầu tư xây dựng các công trình lớn, nhưng phải có đủ năng lực để đảm bảo họ thực thi đúng pháp luật, không làm phá vỡ cảnh quan.Mấy ngày trước tôi đã hỏi một số người dân khu vực sông Hồng, họ đều ủng hộ những ý tưởng của đề án là không xây đô thị sầm uất như Seoul ở Hàn Quốc, mà làm trục không gian xanh.Ngoài Hàn Quốc, trên thế giới, đã có nhiều quốc gia xây dựng thành công khu đô thị hai bên bờ sông như Venice, Paris, Amsterdam... Hà Nội nên học tập những mô hình này một cách có chọn lọc, đảm bảo phù hợp với thực tiễn.Nếu thực hiện được thành công các ý tưởng trong đồ án quy hoạch đô thị sông Hồng lần này, Hà Nội sẽ lập nên "kỳ tích đô thị xanh" thời hiện đại.- Không gian đồ án quy hoạch rất đặc biệt vì nằm trên khu vự ngoài đê. Theo ông, bài toán thoát lũ và xây dựng đô thị hai bên sông cần được giải quyết ra sao?- Như tôi đã đề cập ở trên, mục tiêu chống lũ, chỉnh trị sông Hồng và xây dựng đô thị là bài toán quan trọng thành phố cần phải phối hợp với các bộ, ngành liên quan để tính toán, có giải pháp phù hợp. Ý tưởng của đồ án rất táo bạo khi muốn giải quyết vấn đề căn bản là xây dựng không gian văn hóa xanh và đảm bảo phù hợp với hành lang thoát lũ. Bởi trong hành lang thoát lũ thì không được phép xây dựng công trình kiên cố.Tuy nhiên, tôi cho rằng cần thay đổi quan điểm khu vực hành lang thoát lũ chỉ có một chức năng (đơn năng) này, mà cần vận dụng đa năng, tích hợp nhiều mục đích.Từ khi ba nhà máy thủy điện lớn được xây dựng trên dòng sông Đà (Sơn La, Hòa Bình, Lai Châu), thì việc điều tiết, kiểm soát lũ xuống hạ lưu đã được đảm bảo. Hơn nữa, trong thời đại ứng dụng mạnh mẽ công nghệ thông tin và thích ứng với biến đổi khí hậu hiện nay, thì khu vực hành lang thoát lũ cần được sử dụng linh hoạt. Bây giờ, chúng ta có thể dự báo được khi nào lũ lên, khi nào lũ xuống, nên sẽ chủ động được các biện pháp ứng phó. Trong trường hợp xảy ra lũ lụt lớn hàng trăm năm mới có một lần như nhiều người từng nói, thì ngoài ba cấp đê bảo vệ Hà Nội, vẫn còn chỗ dự trữ thoát hiểm là sông Đáy.Vì vậy, dựa trên các nghiên cứu, dự báo khoa học, thành phố cần thông tin đến người dân và nhà đầu tư rõ ràng về các khu vực có thể xảy ra lũ lụt thường xuyên, không xây dựng công trình kiên cố; khu vực không hoặc ít xảy ra lũ lụt, có thể xây dựng công trình kiên cố. Những khu vực dân đã ở ổn định và đảm bảo an toàn cần cho phép xây dựng.Trước đây, người Hàn Quốc cũng từng đặt mục tiêu chỉnh trị dòng sông Hàn, để xây thành phố Seoul hiện đại với hàng chục triệu dân bên bờ sông. Hà Nội cũng có thể làm nên kỳ tích xây dựng khu đô thị sông Hồng trở thành trục cảnh quan nổi tiếng khu vực và thế giới, nếu thực hiện thành công mục tiêu chỉnh trị dòng sông này.
'''

start_time = time.time()
similar_index_newspapers = get_similar_newspapers(text)
print(f'--- {time.time() - start_time} seconds ---')

print('Number of similar newspapers =', len(similar_index_newspapers))
for index_newspaper in similar_index_newspapers:
    print(newspapers[index_newspaper])

--- 2.135096311569214 seconds ---
Number of similar newspapers = 1
'Ba bài toán' quy hoạch đô thị sông Hồng.txt
